In [11]:
import pandas as pd
from datetime import datetime
import openpyxl as px
 
#プログラム2｜エクセルを読み込む
filepath = 'sample.xlsx'
print(filepath)
 
#プログラム3｜エクセルをpandasで読み込む
df = pd.read_excel(filepath)
df['Date'] = pd.to_datetime(df['Date']).dt.strftime("%Y-%m")

#プログラム4｜「Product」「Area」「Date」のそれぞれの列で重複を削除したリストを作成
products = sorted(list(df['Product'].unique()))
areas = sorted(list(df['Area'].unique()))
dates = sorted(list(df['Date'].unique()))
 
#プログラム5｜保存ファイル名を作成
now = datetime.now()
hiduke = now.strftime('%Y-%m-%d')
 
#プログラム6｜「Product」ごとにピボットテーブルを作成（index=Date, 列=Area, 値=Price(累計値)）
for product in products:
    filtered = df[df['Product'] == f'{product}']
    sales = pd.pivot_table(df, index=filtered['Date'], columns='Area', values='Price', aggfunc='sum',fill_value=0)
    filepath = f'{hiduke}_{product}.xlsx'
    sales.to_excel(filepath,sheet_name='製品売上',startrow=3)
 
    #プログラム7｜プログラム6のエクセルを開く
    wb = px.load_workbook(filepath)
    ws = wb['製品売上']
 
    #プログラム8｜エクセルへ文字列を出力
    ws.cell(row=1, column=1).value = f'{product}_製品売上'
    ws.cell(row=1, column=1).font = px.styles.Font(size=18, bold=True)
    ws.cell(row=2, column=1).value = '月次レポート'
    ws.cell(row=2, column=1).font = px.styles.Font(size=16, bold=True)
 
    #プログラム9｜グラフを生成
    chart = px.chart.LineChart()
    data = px.chart.Reference(ws, min_col=2, max_col=len(areas)+1, min_row=4, max_row=len(dates)+4)
    categories = px.chart.Reference(ws, min_col=1, max_col=1, min_row=5, max_row=len(dates)+4)
    chart.add_data(data, titles_from_data=True)
    chart.set_categories(categories)
 
    #プログラム10｜グラフの表示を整える
    chart.style = 14
    chart.title = '製品売上'
    chart.y_axis.title = '円'
    chart.x_axis.title = '年月'
    chart.height = 9
    chart.width = 16
 
    #プログラム11｜グラフを表示
    ws.add_chart(chart, "G2")
 
    #プログラム12｜エクセルを上書き保存
    wb.save(filepath)


sample.xlsx
